
<center><u><H1>Collaborative Filtering: Model Based</H1></u></center>

In [1]:
import pandas as pd
import numpy as np

In [2]:
header = ['user_id', 'movie_id', 'rating', 'timestamp']
df_ratings = pd.read_csv('../Data/ml-100k/u.data', sep='\t', names=header)
df_ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [30]:
user_data = df_ratings[df_ratings.user_id == 2]
user_data.head(5)

,user_id,movie_id,rating,timestamp
700,2,292,4,888550774
924,2,251,5,888552084
1052,2,50,5,888552084
3425,2,314,1,888980085
5063,2,297,4,888550871


In [31]:
print ('User has already rated {0} movies.'.format(user_data.shape[0]))

User has already rated 62 movies.


In [29]:
header = ['movie_id','movie title','release date','video release date', 'IMDb URL'
          ,'unknown','Action','Adventure','Animation',
              'Childrens','Comedy','Crime','Documentary','Drama','Fantasy',
          'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
          'Thriller','War','Western']
names = ['movie id','movie title']
df_movies = pd.read_csv('../Data/ml-100k/u.item', sep='|',names=header, encoding='ISO-8859-1')
df_movies.head()

,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
cols = ['movie_id', 'title']
movies = pd.read_csv('../Data/ml-100k/u.item', sep='|', names=cols, usecols=range(2), encoding='latin-1')
ratings = pd.merge(movies, df_ratings).sort_values(['rating'], ascending=False)
ratings.head()

,movie_id,title,user_id,rating,timestamp
33615,218,Cape Fear (1991),388,5,886441083
53455,357,One Flew Over the Cuckoo's Nest (1975),532,5,892519935
53463,357,One Flew Over the Cuckoo's Nest (1975),499,5,885599372
31674,208,Young Frankenstein (1974),643,5,891448136
17608,125,Phenomenon (1996),109,5,880564534


In [7]:
matrix_ = df_ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
matrix_.head(5)

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#number of users
len(matrix_)

943

## Matrix Factorization:

In [9]:
from scipy.sparse.linalg import svds

In [10]:
# Example:
myMatrix = np.matrix(np.random.random((5,5)))*5
myMatrix

matrix([[2.95122671, 0.56930686, 1.55267794, 3.52240222, 4.94262647],
        [1.27347411, 3.99953547, 2.00136015, 0.58209253, 4.52201353],
        [2.52290731, 1.73176882, 2.01440109, 1.35663559, 4.5102414 ],
        [0.80468156, 1.09033256, 0.02883424, 1.01181843, 0.26038276],
        [0.23890587, 3.28115131, 2.27689987, 4.10364452, 2.22327194]])

In [11]:
u1, s1, vt1 = svds(myMatrix, k = 4)
s_diag=np.diag(s1)
pred = np.dot(np.dot(u1, s_diag), vt1)
df_pred = pd.DataFrame(pred)
df_pred

,0,1,2,3,4
0,3.044419,0.553917,1.722170,3.489768,4.849167
1,1.362403,3.984850,2.163099,0.550951,4.432829
2,2.357744,1.759043,1.714013,1.414473,4.675878
3,0.793403,1.092195,0.008322,1.015768,0.271694
4,0.207520,3.286334,2.219818,4.114635,2.254747


In [12]:
matrix = matrix_.values

In [13]:
user_ratings_mean = np.mean(matrix, axis = 1)

In [14]:
matrix_demeaned = matrix - user_ratings_mean.reshape(-1, 1)

In [15]:
matrix_demeaned[:10,:10]

array([[ 4.41617122,  2.41617122,  3.41617122,  2.41617122,  2.41617122,
         4.41617122,  3.41617122,  0.41617122,  4.41617122,  2.41617122],
       [ 3.86325803, -0.13674197, -0.13674197, -0.13674197, -0.13674197,
        -0.13674197, -0.13674197, -0.13674197, -0.13674197,  1.86325803],
       [-0.08977408, -0.08977408, -0.08977408, -0.08977408, -0.08977408,
        -0.08977408, -0.08977408, -0.08977408, -0.08977408, -0.08977408],
       [-0.06183115, -0.06183115, -0.06183115, -0.06183115, -0.06183115,
        -0.06183115, -0.06183115, -0.06183115, -0.06183115, -0.06183115],
       [ 3.70095125,  2.70095125, -0.29904875, -0.29904875, -0.29904875,
        -0.29904875, -0.29904875, -0.29904875, -0.29904875, -0.29904875],
       [ 3.54399524, -0.45600476, -0.45600476, -0.45600476, -0.45600476,
        -0.45600476,  1.54399524,  3.54399524,  3.54399524, -0.45600476],
       [-0.95005945, -0.95005945, -0.95005945,  4.04994055, -0.95005945,
        -0.95005945,  4.04994055,  4.04994055

In [16]:
#get SVD components from train matrix. Choose k.
u, s, vt = svds(matrix_demeaned, k = 100)
s_diag_matrix=np.diag(s)

In [17]:
all_user_pred_ratings = np.dot(np.dot(u, s_diag_matrix), vt) + user_ratings_mean.reshape(-1, 1)

In [18]:
df_predictions = pd.DataFrame(all_user_pred_ratings, columns=matrix_.columns)
df_predictions.head(10)

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,5.219037,3.216264,2.330048,4.070080,2.174042,2.371151,4.937889,2.451562,5.018531,2.214036,...,0.019273,-0.003311,-0.055446,-0.036462,0.027922,0.005369,0.013090,0.009229,0.034662,0.176596
1,2.546173,-0.189924,0.398652,0.136218,0.219939,0.390755,0.373605,-0.107817,1.347373,1.063680,...,0.027430,0.013848,-0.032258,-0.026604,-0.027045,-0.012007,-0.005429,-0.008718,0.006852,-0.028813
2,-0.145042,-0.101736,-0.411722,-0.288039,0.016932,-0.026025,-0.318215,0.120447,-0.243985,0.198292,...,-0.034298,-0.053417,0.022753,0.004712,-0.030537,-0.012318,0.025788,0.006735,-0.036110,-0.058119
3,0.258836,-0.191892,-0.029108,0.068951,0.140218,0.109049,0.222686,-0.359241,0.014793,-0.092021,...,0.001857,0.016633,0.016123,0.010074,0.006319,0.008731,0.030237,0.019484,-0.004342,-0.033769
4,4.200703,1.782011,0.068438,0.170432,0.266356,0.114757,1.386346,-0.020448,-0.302272,0.669130,...,0.054404,-0.051297,-0.025636,-0.018048,-0.065828,-0.010397,-0.025445,-0.017921,-0.012638,-0.071162
5,4.764227,0.230788,0.162792,0.173629,0.276525,0.234543,1.676095,3.988945,4.539381,-0.100202,...,0.017533,0.074215,-0.015597,0.001958,0.222465,0.036429,0.035147,0.035788,-0.022903,-0.000419
6,-0.385357,-0.029964,-0.240016,4.358223,0.612881,-0.189489,5.716326,3.892249,6.794257,4.487480,...,0.106762,0.067191,0.053888,0.045989,0.086807,0.033797,0.041011,0.037404,-0.041297,0.057709
7,1.163665,0.982958,-0.137211,1.706218,-0.240101,-0.129168,1.976962,0.671966,-0.347451,0.213261,...,-0.031171,-0.035149,0.042364,0.031954,0.019395,0.011621,0.012594,0.012107,0.048017,-0.026221
8,1.114760,-0.173776,0.177938,-0.083087,0.365881,0.296733,1.317335,-0.435066,1.036750,-0.040463,...,0.011828,-0.022435,0.013152,0.011422,-0.020182,0.009450,0.012428,0.010939,-0.041944,0.044579
9,3.346179,0.041667,0.726061,3.647356,-0.531409,-0.071195,4.919051,2.946155,3.894497,0.355635,...,0.051248,-0.048035,0.125531,0.087138,0.078619,-0.000718,-0.022861,-0.011789,-0.028649,-0.031692


## Top 20 movies predictions for a user

In [19]:
sorted_user_predictions = df_predictions.iloc[1].sort_values(ascending=False)
sorted_user_predictions[:20]

movie_id
286    5.675194
127    5.151598
275    4.933607
302    4.709105
269    4.575182
100    4.293844
285    4.202150
300    4.067023
50     4.001802
313    3.941593
257    3.770242
272    3.492483
301    3.437639
237    3.428083
242    3.386335
289    3.336877
303    3.288627
14     3.207575
283    3.165430
306    3.058510
Name: 1, dtype: float64

## Calculating Metrics (RMSE):

In [20]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [21]:
def rmse(pred,actual):
    prediction = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, actual))

In [22]:
pred_ratings = np.dot(np.dot(u, s_diag_matrix), vt)

In [23]:
#print "User-based CF MSE:"
str(rmse(pred_ratings, matrix))

'1.7757864359959719'

## Sparsity level:

### Sparsity describes the percentage of cells in a database table that are not populated.

In [24]:
n_users = df_ratings.user_id.unique().shape[0]
n_items = df_ratings.movie_id.unique().shape[0]

In [25]:
sparsity=round(1.0-len(df_ratings)/float(n_users*n_items),3)
print('The sparsity level is:' + str(sparsity*100)+'%')

The sparsity level is:93.7%


## References:

https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.sparse.linalg.svds.html

https://grouplens.org/datasets/movielens/latest/